In [ ]:
# install libraries
!pip install python-dotenv
!pip install langchain

In [1]:
# Import necessary libraries
from dotenv import load_dotenv
from langchain.agents import Tool, initialize_agent
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
import os

In [ ]:
# Load environment variables from .env file
load_dotenv()

In [3]:
# create the prompt template to the tool
execute_task_prompt = PromptTemplate(
    template="""Given the following overall question `{input}`.

    Perform the task by understanding the problem, extracting variables, and being smart
    and efficient. Write a detailed response that address the task.
    When confronted with choices, make a decision yourself with reasoning.
    """,
    input_variables=["input"],
)

In [10]:
# Retrieve API key and DaniGPT agent ID from environment variables
codegpt_api_key= os.getenv("CODEGPT_API_KEY")
code_gpt_agent_id= os.getenv("CODEGPT_AGENT_ID")
codegpt_api_base = "https://api.codegpt.co/v1"

In [5]:
# Create a ChatOpenAI object with the retrieved API key, API base URL, and agent ID
tool_agent_llm = ChatOpenAI(openai_api_key=codegpt_api_key,
                openai_api_base=codegpt_api_base,
                model=code_gpt_agent_id, verbose=True)
llm_chain = LLMChain(llm=tool_agent_llm, prompt=execute_task_prompt)

In [ ]:
# create and add the new llm tool
danigpt_tool = Tool(
    name='DaniGPT',
    func=llm_chain.run,
    description="Useful for when you need to answer questions about Judini"
) 
tools = [danigpt_tool]

In [7]:
#memory
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=3,
    return_messages=True
)

# general llm agent
llm_openai = ChatOpenAI(model="gpt-4", temperature=0)

In [11]:
agent = initialize_agent(
                agent="chat-conversational-react-description",
                tools=tools,
                llm=llm_openai,
                verbose=True,
                max_iteration=3,
                early_stop_method="generate",
                memory=memory
            )

In [ ]:
print(agent.run("what is Judini?"))